In [1]:
import pandas as pd
from transformers import pipeline
import re
from tqdm.notebook import tqdm
import duckdb

In [2]:
# TRAER MODELO A ESPAÑOL

modelo = "nlptown/bert-base-multilingual-uncased-sentiment"
clasificador = pipeline("sentiment-analysis", model=modelo)

Device set to use cpu


In [3]:
# PALABRAS NEGATIVAS
def clasificar_texto_reglas(texto):
    if pd.isna(texto):
        return None
    
    texto_l = str(texto).lower().strip()
    
    # Textos vacíos o inválidos → RETORNAR INMEDIATAMENTE
    if texto_l in ["", "na", "n/a", "null", "ninguna", "ninguno", "."]:
        return "VACIO"  # Marcador especial
    
    palabras_negativas = [
        "aburrid", "mediocre", "malo", "mal", "pésim", "flojo",
        "no sirve", "no explic", "no aprend", "pérdida de tiempo",
        "perder el tiempo", "solo habla", "no veo la necesidad",
        "no aporta", "muy mal", "desorganizado", "caótic"
    ]
    palabras_positivas = [
        "excelente", "muy bueno", "buen profesor", "explica bien",
        "me gustó", "me gusto", "clases buenas", "genial", "recomendado"
    ]
    
    for p in palabras_negativas:
        if p in texto_l:
            return 1   # NEGATIVO
    
    for p in palabras_positivas:
        if p in texto_l:
            return 5   # POSITIVO
    
    return None  # No aplica regla → usar modelo

In [4]:
# FUNCION PARA ETIQUETAR 
def interpretar_label(label, classifier):
    if not isinstance(label, str):
        return None
    
    lab = label.strip()
    
    # Manejo del modelo 'nlptown/bert-base-multilingual-uncased-sentiment'
    # Devuelve etiquetas como '1 star', '2 stars', ..., '5 stars'
    if "star" in lab.lower():
        if lab.startswith("1"):
            return 1   # NEGATIVO
        if lab.startswith("2"):
            return 1   # también negativo
        if lab.startswith("3"):
            return 3   # REGULAR
        if lab.startswith("4"):
            return 5   # POSITIVO
        if lab.startswith("5"):
            return 5   # POSITIVO
        return None
    
    # Manejo de modelos tipo BETO (POS/NEG/NEU)
    if lab.upper().startswith("LABEL_"):
        try:
            idx = int(lab.split("_")[-1])
            id2label = classifier.model.config.id2label
            lab = id2label.get(idx, lab)
        except Exception:
            pass
    
    lab_clean = re.sub(r'[^A-Za-z]', '', lab).lower()
    
    if lab_clean.startswith("pos"):
        return 5
    if lab_clean.startswith("neg"):
        return 1
    if lab_clean.startswith("neu") or lab_clean == "neutral":
        return 3
    if lab.upper().startswith("POS"):
        return 5
    if lab.upper().startswith("NEG"):
        return 1
    
    return None

In [5]:
# FUNCION PRINCIPAL CON EL ANÁLISIS DE SENTIMIENTOS

def clasificar_sentimiento(texto):
    if pd.isna(texto):
        return None

    texto_limpio = str(texto).strip().lower()
    if texto_limpio in ["", "na", "n/a", "null", "ninguna", "ninguno","."]:
        return None  # INDETERMINADO

    # 1. Reglas primero
    regla = clasificar_texto_reglas(texto)
    if regla is not None:
        return regla

    # 2. Si no aplica regla, usar modelo
    resultado = clasificador(texto)[0]
    valor = interpretar_label(resultado["label"], clasificador)
    return valor

In [6]:
# PROCESAR HOJAS 
def procesar_hoja(df):
    mask = df["pre_num"] == 20
    df_20 = df[mask].copy()
    
    textos = df_20["respuesta"].fillna("").astype(str).tolist()
    resultados_finales = [None] * len(textos)  # Pre-asignar lista
    
    batch_size = 64
    
    # Barra de progreso
    for i in tqdm(range(0, len(textos), batch_size), desc="Procesando textos", ncols=80):
        batch = textos[i:i+batch_size]
        
        # 1. Aplicar reglas
        reglas = [clasificar_texto_reglas(t) for t in batch]
        
        # 2. Filtrar textos que necesitan modelo (excluir "VACIO")
        indices_modelo = [j for j in range(len(batch)) if reglas[j] is None]
        textos_modelo = [batch[j] for j in indices_modelo]
        
        # 3. Pasar al modelo solo los que lo requieren
        resultados_modelo = {}
        if textos_modelo:
            out_modelo = clasificador(textos_modelo, truncation=True)
            for idx, resultado in zip(indices_modelo, out_modelo):
                numero = interpretar_label(resultado["label"], clasificador)
                resultados_modelo[idx] = numero
        
        # 4. Mezclar reglas + modelo en orden correcto
        for j in range(len(batch)):
            pos_global = i + j
            
            if reglas[j] == "VACIO":
                # Textos vacíos → dejar como None (NaN en pandas)
                resultados_finales[pos_global] = None
            elif reglas[j] is not None:
                resultados_finales[pos_global] = reglas[j]
            elif j in resultados_modelo:
                resultados_finales[pos_global] = resultados_modelo[j]
            else:
                resultados_finales[pos_global] = None
    
    # Asignar resultados al dataframe original
    df.loc[mask, "opcion"] = resultados_finales
    
    return df

In [7]:
# LEER ARCHIVOS

ruta = r"C:\Users\9 ----- SIG\Downloads\HETEROEVALUACION.xlsx"

df1 = duckdb.sql("""
    SELECT *
    EXCLUDE (opc_calificacion, pre_papa, pre_numeral, nombre_quien, enc_id, carrera_para,sede_para)
    FROM 'C:/Users/9 ----- SIG/Downloads/HETEROEVALUACION.xlsx'
""").df() #lectura autoevaluacion parte 1 muy pesado
df2 = pd.read_excel(ruta, sheet_name=1)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
# ELIMINAR COLUMNAS DE LA HOJA 2 INNECESARIAS

cols = [
    "opc_calificacion", "pre_papa", "pre_numeral", "nombre_quien",
    "nombre_quien", "sede_para", "carrera_para", "enc_id"
]

df2 = df2.drop(columns=cols, errors="ignore")

In [9]:
# INICIAR PROCESO

df1 = procesar_hoja(df1)
df2 = procesar_hoja(df2)

Procesando textos:   0%|                                | 0/782 [00:00<?, ?it/s]

Procesando textos:   0%|                                | 0/145 [00:00<?, ?it/s]

C:\Users\9 ----- SIG\AppData\Local\Temp\ipykernel_1036\1873385664.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[None, None, None, None, 1, None, 3, None, 5, None, None, None, None, 5, 1, 1, None, None, None, 5, None, 1, 5, None, 5, None, None, None, None, None, None, 5, 5, 5, 5, 3, None, None, 1, None, 5, 5, None, 3, None, None, 5, 5, None, 5, 1, None, 1, 5, 5, 5, 5, None, 5, None, None, 5, 5, None, 5, None, None, 5, None, 5, 5, None, None, 5, None, 5, None, 1, 5, 5, None, 5, None, 5, 5, 5, None, 5, 1, None, 5, None, 5, 1, 1, None, 1, None, 5, 1, None, 5, None, None, None, 5, 3, None, 3, 1, 5, None, 5, None, None, 1, 1, 1, None, 5, None, None, 3, None, None, 1, 5, 5, None, None, 5, 1, None, None, 1, 1, 1, None, 5, None, 5, None, 5, None, None, 1, None, 5, None, 1, 5, None, 5, 5, None, 5, None, 5, 1, None, None, None, 5, None, 1, None, None, 1, None, 1, 5, 5, None, 3, None, None, None, 5, 1, 1, No

In [10]:
# Guardar en archivo con nombre "h1"
ruta_salida = r"C:\Users\9 ----- SIG\Downloads\hetero_con_analisis.xlsx"

with pd.ExcelWriter(ruta_salida, engine='openpyxl') as writer:
    df1.to_excel(writer, sheet_name='hoja1', index=False)
    df2.to_excel(writer, sheet_name='hoja2', index=False)

print(f"Archivo guardado en: {ruta_salida}")

Archivo guardado en: C:\Users\9 ----- SIG\Downloads\hetero_con_analisis.xlsx
